In [4]:
import pandas as pd

In [5]:
%reload_kedro
df = catalog.load("preprocessed_udacity")
df.head()

2021-01-03 17:04:50,622 - root - INFO - ** Kedro project coursing-ml
2021-01-03 17:04:50,624 - root - INFO - Defined global variable `context` and `catalog`
2021-01-03 17:04:50,626 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_udacity` (CSVDataSet)...


,collaboration,description,difficulty,duration,free,id_course,n_reviews,rating,school,skills,title,url,time_range
0,Insight,"Learn to design data models, build data wareho...",intermediate,200.0,0,nd027,918,91.089325,School of Data Science,"Data Modeling, Data Pipelines, Data Lakes, Spa...",Become a Data Engineer,https://www.udacity.com/course/data-engineer-n...,months
1,"Google, Hootsuite, HubSpot, MailChimp, Moz, Fa...","Learn to create marketing content, use social ...",beginner,120.0,0,nd018,1753,93.964632,School of Business,"Digital Marketing, Social Media Marketing, Fac...",Become a Digital Marketer,https://www.udacity.com/course/digital-marketi...,months
2,NaN,You’ll learn how to create a digital user expe...,beginner,120.0,0,nd578,619,94.248788,School of Business,"User Research, User Experience, Prototyping, I...",Become a UX Designer,https://www.udacity.com/course/ux-designer-nan...,months
3,"AWS, Facebook Artificial Intelligence","Become an expert in neural networks, and learn...",intermediate,160.0,0,nd101,2893,93.218113,School of Artificial Intelligence,"Deep Learning, Neural Networks, Jupyter Notebo...",Deep Learning,https://www.udacity.com/course/deep-learning-n...,months
4,NaN,The goal of the Full Stack Web Developer Nanod...,intermediate,160.0,0,nd0044,517,90.870406,School of Programming & Development,"Python, JavaScript, PostgreSQL, Flask, Docker,...",Full Stack Web Developer,https://www.udacity.com/course/full-stack-web-...,months


# Missing Values

In [6]:
def check_missing(data,output_path=None):
    """
    check the total number & percentage of missing values
    per variable of a pandas Dataframe
    """
    
    result = pd.concat([data.isnull().sum(),data.isnull().mean()],axis=1)
    result = result.rename(index=str,columns={0:'total missing',1:'proportion'})
    return result

c:\users\enrmeao\pycharmprojects\coursing\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
check_missing(data=df)

,total missing,proportion
collaboration,111,0.444
description,11,0.044
difficulty,6,0.024
duration,5,0.020
free,0,0.000
id_course,0,0.000
n_reviews,0,0.000
rating,200,0.800
school,0,0.000
skills,16,0.064


## Rellenar manualmente

In [8]:
mask = df["title"] == 'Shell Workshop'
df.loc[mask, "difficulty"] = 'beginner'

c:\users\enrmeao\pycharmprojects\coursing\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
# porque la mayoría de cursos con este parámetro a null son de nivel intermedio
mask = df["difficulty"].isnull()
df.loc[mask, "difficulty"] = 'intermediate' 

In [10]:
check_missing(data=df)

,total missing,proportion
collaboration,111,0.444
description,11,0.044
difficulty,0,0.000
duration,5,0.020
free,0,0.000
id_course,0,0.000
n_reviews,0,0.000
rating,200,0.800
school,0,0.000
skills,16,0.064


In [11]:
mask = df["duration"].isnull()
df.loc[mask, "duration"] = 10

c:\users\enrmeao\pycharmprojects\coursing\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Arbitrary Value Imputation

In [12]:
df["collaboration"].fillna("", inplace=True)

In [13]:
df["description"].fillna("", inplace=True)

In [14]:
df["skills"].fillna("", inplace=True)

In [ ]:
df["title"].fillna("", inplace=True)

In [15]:
check_missing(data=df)

,total missing,proportion
collaboration,0,0.0
description,0,0.0
difficulty,0,0.0
duration,0,0.0
free,0,0.0
id_course,0,0.0
n_reviews,0,0.0
rating,200,0.8
school,0,0.0
skills,0,0.0


In [17]:
df.to_csv('output_cleaned.csv')

# Outliers

In [16]:
import pandas as pd
import numpy as np
# from warnings import warn

# 2018.11.07 Created by Eamon.Zhang

def outlier_detect_arbitrary(data,col,upper_fence,lower_fence):
    '''
    identify outliers based on arbitrary boundaries passed to the function.
    '''

    para = (upper_fence, lower_fence)
    tmp = pd.concat([data[col]>upper_fence,data[col]<lower_fence],axis=1)
    outlier_index = tmp.any(axis=1)
    print('Num of outlier detected:',outlier_index.value_counts()[1])
    print('Proportion of outlier detected',outlier_index.value_counts()[1]/len(outlier_index))    
    return outlier_index, para



def outlier_detect_IQR(data,col,threshold=3):
    '''
    outlier detection by Interquartile Ranges Rule, also known as Tukey's test. 
    calculate the IQR ( 75th quantile - 25th quantile) 
    and the 25th 75th quantile. 
    Any value beyond:
        upper bound = 75th quantile + （IQR * threshold）
        lower bound = 25th quantile - （IQR * threshold）   
    are regarded as outliers. Default threshold is 3.
    '''
     
    IQR = data[col].quantile(0.75) - data[col].quantile(0.25)
    Lower_fence = data[col].quantile(0.25) - (IQR * threshold)
    Upper_fence = data[col].quantile(0.75) + (IQR * threshold)
    para = (Upper_fence, Lower_fence)
    tmp = pd.concat([data[col]>Upper_fence,data[col]<Lower_fence],axis=1)
    outlier_index = tmp.any(axis=1)
    print('Num of outlier detected:',outlier_index.value_counts()[1])
    print('Proportion of outlier detected',outlier_index.value_counts()[1]/len(outlier_index))
    return outlier_index, para


def outlier_detect_mean_std(data,col,threshold=3):
    '''
    outlier detection by Mean and Standard Deviation Method.
    If a value is a certain number(called threshold) of standard deviations away 
    from the mean, that data point is identified as an outlier. 
    Default threshold is 3.
    This method can fail to detect outliers because the outliers increase the standard deviation. 
    The more extreme the outlier, the more the standard deviation is affected.
    '''
   
    Upper_fence = data[col].mean() + threshold * data[col].std()
    Lower_fence = data[col].mean() - threshold * data[col].std()   
    para = (Upper_fence, Lower_fence)   
    tmp = pd.concat([data[col]>Upper_fence,data[col]<Lower_fence],axis=1)
    outlier_index = tmp.any(axis=1)
    print('Num of outlier detected:',outlier_index.value_counts()[1])
    print('Proportion of outlier detected',outlier_index.value_counts()[1]/len(outlier_index))
    return outlier_index, para


def outlier_detect_MAD(data,col,threshold=3.5):
    """
    outlier detection by Median and Median Absolute Deviation Method (MAD)
    The median of the residuals is calculated. Then, the difference is calculated between each historical value and this median. 
    These differences are expressed as their absolute values, and a new median is calculated and multiplied by 
    an empirically derived constant to yield the median absolute deviation (MAD). 
    If a value is a certain number of MAD away from the median of the residuals, 
    that value is classified as an outlier. The default threshold is 3 MAD.
    
    This method is generally more effective than the mean and standard deviation method for detecting outliers, 
    but it can be too aggressive in classifying values that are not really extremely different. 
    Also, if more than 50% of the data points have the same value, MAD is computed to be 0, 
    so any value different from the residual median is classified as an outlier.
    """
    
    median = data[col].median()
    median_absolute_deviation = np.median([np.abs(y - median) for y in data[col]])
    modified_z_scores = pd.Series([0.6745 * (y - median) / median_absolute_deviation for y in data[col]])
    outlier_index = np.abs(modified_z_scores) > threshold
    print('Num of outlier detected:',outlier_index.value_counts()[1])
    print('Proportion of outlier detected',outlier_index.value_counts()[1]/len(outlier_index))
    return outlier_index

    
    
def windsorization(df,col,bounds,strategy='both'):
    """
    top-coding & bottom coding (capping the maximum of a distribution at an arbitrarily set value,vice versa)
    """
    if strategy == 'both':
        df.loc[df[col]>bounds[1],col] = bounds[1]
        df.loc[df[col]<bounds[0],col] = bounds[0]
    elif strategy == 'top':
        df.loc[df[col]>bounds[1],col] = bounds[1]
    elif strategy == 'bottom':
        df.loc[df[col]<bounds[0],col] = bounds[0]  


def drop_outlier(data,outlier_index):
    """
    drop the cases that are outliers
    """
    
    data_copy = data[~outlier_index]
    return data_copy


def impute_outlier_with_avg(data,col,outlier_index,strategy='mean'):
    """
    impute outlier with mean/median/most frequent values of that variable.
    """
    
    data_copy = data.copy(deep=True)
    if strategy=='mean':
        data_copy.loc[outlier_index,col] = data_copy[col].mean()
    elif strategy=='median':
        data_copy.loc[outlier_index,col] = data_copy[col].median()
    elif strategy=='mode':
        data_copy.loc[outlier_index,col] = data_copy[col].mode()[0]   
        
    return data_copy

c:\users\enrmeao\pycharmprojects\coursing\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [101]:
### Duration

In [102]:
index,para = outlier_detect_mean_std(data=df,col='duration',threshold=3)
print('Upper bound:',para[0],'\nLower bound:',para[1])

Num of outlier detected: 1
Proportion of outlier detected 0.004
Upper bound: 249.6656767753521 
Lower bound: -101.97767677535211


In [103]:
# check the 20 found outliers
df.loc[index,'duration'].sort_values()

c:\users\enrmeao\pycharmprojects\coursing\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


241    280.0
Name: duration, dtype: float64

In [104]:
df.iloc[241]

collaboration                                                 None
description      You will start by learning the basics of iOS a...
difficulty                                            intermediate
duration                                                       280
free                                                             0
id_course                                                    nd003
n_reviews                                                      765
rating                                                      92.366
school                         School of Programming & Development
skills           Swift programming, Core Data, UIKit, iOS netwo...
title                                      Become an iOS Developer
url              https://www.udacity.com/course/ios-developer-n...
time_range                                                  months
Name: 241, dtype: object

In [105]:
index = outlier_detect_MAD(data=df,col='duration',threshold=3.5)

Num of outlier detected: 1
Proportion of outlier detected 0.004


In [106]:
df.loc[index,'duration'].sort_values()

c:\users\enrmeao\pycharmprojects\coursing\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


241    280.0
Name: duration, dtype: float64

In [107]:
### Número de reviews

In [113]:
index,para = outlier_detect_mean_std(data=df,col='n_reviews',threshold=3)
print('Upper bound:',para[0],'\nLower bound:',para[1])

Num of outlier detected: 7
Proportion of outlier detected 0.028
Upper bound: 1054.913524769306 
Lower bound: -828.465524769306


c:\users\enrmeao\pycharmprojects\coursing\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [114]:
df[index]

c:\users\enrmeao\pycharmprojects\coursing\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,collaboration,description,difficulty,duration,free,id_course,n_reviews,rating,school,skills,title,url,time_range
1,"Google, Hootsuite, HubSpot, MailChimp, Moz, Fa...","Learn to create marketing content, use social ...",beginner,120.0,0,nd018,1700,93.964632,School of Business,"Digital Marketing, Social Media Marketing, Fac...",Become a Digital Marketer,https://www.udacity.com/course/digital-marketi...,months
3,"AWS, Facebook Artificial Intelligence","Become an expert in neural networks, and learn...",intermediate,160.0,0,nd101,1700,93.218113,School of Artificial Intelligence,"Deep Learning, Neural Networks, Jupyter Notebo...",Deep Learning,https://www.udacity.com/course/deep-learning-n...,months
5,None,"Learn the basics of programming through HTML, ...",beginner,160.0,0,nd000,1700,93.642036,School of Programming & Development,"Introduction to Programming, HTML, CSS, Python",Learn to Code,https://www.udacity.com/course/intro-to-progra...,months
8,"Tableau, Mode","In this program, you’ll learn foundational dat...",beginner,120.0,0,nd098,1161,95.021533,School of Business,"Excel & Spreadsheets, SQL, Data Visualization,...",Business Analytics,https://www.udacity.com/course/business-analyt...,months
10,Kaggle,Advance your programming skills and refine you...,intermediate,160.0,0,nd002,1700,92.290345,School of Data Science,"Data Wrangling, Matplotlib, Bootstrapping, Pan...",Become a Data Analyst,https://www.udacity.com/course/data-analyst-na...,months
14,"Mercedes-Benz, Nvidia, Uber ATG, DiDi, McLaren",You'll first apply computer vision and deep le...,advanced,240.0,0,nd013,1386,95.050505,School of Autonomous Systems,"Computer Vision, Deep Learning, Sensor Fusion,...",Become a Self-Driving Car Engineer,https://www.udacity.com/course/self-driving-ca...,months
234,Google,We built this curriculum with Google for aspir...,beginner,160.0,0,nd803,1700,96.822186,School of Programming & Development,"Android Development, Java programming, Android...",Android Basics by Google,https://www.udacity.com/course/android-basics-...,months


In [110]:
index = outlier_detect_MAD(data=df,col='n_reviews',threshold=3.5)

Num of outlier detected: 50
Proportion of outlier detected 0.2


c:\users\enrmeao\pycharmprojects\coursing\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
c:\users\enrmeao\pycharmprojects\coursing\venv\lib\site-packages\ipykernel_launcher.py:80: RuntimeWarning: divide by zero encountered in double_scalars
c:\users\enrmeao\pycharmprojects\coursing\venv\lib\site-packages\ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in double_scalars


MAD es muy agresivo, detecta un 20% de anomalías

### Windsorization

Esto mejor dejarlo para más adelante

In [111]:
windsorization(df=df,col='n_reviews',bounds=[0,1700],strategy='top')

c:\users\enrmeao\pycharmprojects\coursing\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [112]:
df

c:\users\enrmeao\pycharmprojects\coursing\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,collaboration,description,difficulty,duration,free,id_course,n_reviews,rating,school,skills,title,url,time_range
0,Insight,"Learn to design data models, build data wareho...",intermediate,200.0,0,nd027,918,91.089325,School of Data Science,"Data Modeling, Data Pipelines, Data Lakes, Spa...",Become a Data Engineer,https://www.udacity.com/course/data-engineer-n...,months
1,"Google, Hootsuite, HubSpot, MailChimp, Moz, Fa...","Learn to create marketing content, use social ...",beginner,120.0,0,nd018,1700,93.964632,School of Business,"Digital Marketing, Social Media Marketing, Fac...",Become a Digital Marketer,https://www.udacity.com/course/digital-marketi...,months
2,None,You’ll learn how to create a digital user expe...,beginner,120.0,0,nd578,619,94.248788,School of Business,"User Research, User Experience, Prototyping, I...",Become a UX Designer,https://www.udacity.com/course/ux-designer-nan...,months
3,"AWS, Facebook Artificial Intelligence","Become an expert in neural networks, and learn...",intermediate,160.0,0,nd101,1700,93.218113,School of Artificial Intelligence,"Deep Learning, Neural Networks, Jupyter Notebo...",Deep Learning,https://www.udacity.com/course/deep-learning-n...,months
4,None,The goal of the Full Stack Web Developer Nanod...,intermediate,160.0,0,nd0044,517,90.870406,School of Programming & Development,"Python, JavaScript, PostgreSQL, Flask, Docker,...",Full Stack Web Developer,https://www.udacity.com/course/full-stack-web-...,months
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,"Kaggle, AWS",Learn foundational machine learning algorithms...,intermediate,120.0,0,nd229,342,94.853801,School of Artificial Intelligence,"Introduction to Machine Learning, Supervised L...",Intro to Machine Learning with PyTorch,https://www.udacity.com/course/intro-to-machin...,months
246,None,Build sustainable profit streams from products...,intermediate,40.0,0,nd037-3,0,NaN,School of Business,"Product Management, Monetization Models, Prici...",Monetization Strategy Course,https://www.udacity.com/course/monetization-st...,month
247,WorldQuant,"Learn the basics of quantitative analysis, inc...",intermediate,240.0,0,nd880,515,91.611650,School of Artificial Intelligence,"Momentum Trading Strategy, Smart Beta, Alpha F...",Artificial Intelligence for Trading,https://www.udacity.com/course/ai-for-trading-...,months
248,None,Learn the essential foundations of AI: the pro...,beginner,120.0,0,nd089,839,92.467223,School of Artificial Intelligence,"Python, NumPy, Pandas, Matplotlib, PyTorch",AI Programming with Python,https://www.udacity.com/course/ai-programming-...,months


# Rare values

https://github.com/Yimeng-Zhang/feature-engineering-and-feature-selection/blob/master/2.3_Demo_Rare_Values.ipynb

Causan overfitting en tree-based methods

In [116]:
df.school.value_counts()

School of Programming & Development    139
School of Artificial Intelligence       40
School of Business                      26
School of Data Science                  15
Career Advancement                      12
School of Autonomous Systems            10
School of Cloud Computing                8
Name: school, dtype: int64

In [119]:
print(df.school.value_counts()/len(df))

School of Programming & Development    0.556
School of Artificial Intelligence      0.160
School of Business                     0.104
School of Data Science                 0.060
Career Advancement                     0.048
School of Autonomous Systems           0.040
School of Cloud Computing              0.032
Name: school, dtype: float64


# High Cardinality